In [24]:
# Step 1: Install required libraries
!apt install ffmpeg

# Step 2: Import required libraries
import cv2
import imageio
import numpy as np
import tensorflow as tf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [25]:
# Step 3: Define video paths
input_video_path = '/kaggle/input/videos/0ae2db4ab2a7b56572db4ca3585fcba5.mp4'  
output_video_path = '/kaggle/working/BDD100k_output_video.mp4' 

In [26]:
# Step 4: Open the input video
cap = cv2.VideoCapture(input_video_path)

In [27]:
# Check if the video file is successfully opened
if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

In [28]:
# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video Properties: Width={frame_width}, Height={frame_height}, FPS={fps}")


Video Properties: Width=1920, Height=1080, FPS=25


In [29]:
# Step 5: Initialize the video writer
writer = imageio.get_writer(output_video_path, fps=fps)

In [30]:
from tensorflow.keras.models import load_model

def load_pretrained_model(model_path):

  def dice_loss(y_true, y_pred, smooth=1e-6):
      y_true_f = tf.keras.backend.flatten(y_true)
      y_pred_f = tf.keras.backend.flatten(y_pred)
      intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
      dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
      return 1 - dice

  new_model = tf.keras.models.load_model(
      model_path,
      custom_objects={'dice_loss': dice_loss},
      safe_mode=False  # Allow lambda loading
  )

  return new_model

# Load the model with safe_mode=False
model = load_pretrained_model('/kaggle/input/day_time_model/keras/default/1/Xception_unet_3.keras')


In [31]:
# Step 6: Define preprocess_frame function
def preprocess_frame(frame, target_size):
    """
    Preprocesses a video frame to match the model's input size.
    - Resizes the frame to target_size
    - Normalizes pixel values to [0, 1]
    - Adds a batch dimension
    """
    frame_resized = cv2.resize(frame, target_size)  # Resize frame
    frame_normalized = frame_resized / 255.0  # Normalize pixel values
    return np.expand_dims(frame_normalized, axis=0)  # Add batch dimension

# Define the input size expected by the model
input_size = (256, 256)  # Replace with your model's input dimensions


In [32]:
# Step 8: Process video frames in a loop
while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # Break the loop if no frame is returned
    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_frame(frame, input_size)

    # Make a prediction
    prediction = model.predict(preprocessed_frame)

    # Ensure prediction is a NumPy array
    prediction = np.array(prediction)  # Shape: (1, 256, 256, 1)

    # Process the prediction: Aggregate or summarize
    average_confidence = np.mean(prediction)  # Aggregate confidence
    label = 1 if average_confidence > 0.5 else 0  # Example thresholding for binary classification

    # Add text overlay with the prediction
    text = f"Class: {label}, Confidence: {average_confidence:.2f}"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Optionally, visualize the segmentation map (rescaled to frame size)
    segmentation_map = (prediction[0] * 255).astype('uint8')  # Rescale to [0, 255]
    segmentation_map = cv2.resize(segmentation_map, (frame.shape[1], frame.shape[0]))  # Match original frame size
    overlay = cv2.addWeighted(frame, 0.7, cv2.cvtColor(segmentation_map, cv2.COLOR_GRAY2BGR), 0.3, 0)

    # Write the frame with overlay to the output video
    writer.append_data(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB

print("Video processing complete.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


TypeError: TiffWriter.write() got an unexpected keyword argument 'fps'

In [ ]:
# Step 9: Release resources
cap.release()
writer.close()
print(f"Processed video saved at: {output_video_path}")